In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
from datetime import datetime
import time,re
import pandas as pd
import chromedriver_binary
import random
def random_num(max_num):
    return random.randint(2,max_num)

In [3]:
username="############"
keypass="############"

In [28]:
#DBログイン
browser = webdriver.Chrome()

browser.get("https://app.musubu.in/list/companies")
browser.maximize_window()
browser.find_element_by_xpath('//*[@id="root"]/div/div[2]/div[1]/form/p[1]/input').send_keys(username)
browser.find_element_by_xpath('//*[@id="root"]/div/div[2]/div[1]/form/p[2]/input').send_keys(keypass)
browser.find_element_by_xpath('//*[@id="loginBtn"]').click()
time.sleep(2)
browser.find_element_by_xpath('//*[@id="modal-root"]/div[1]/div/div/div[1]').click()
time.sleep(2)


#要検索企業リストを導入
keylist=["ライブ配信サービス運営",
"ビデオ会議システム開発",
"監視カメラシステム開発",
"IP監視カメラシステム販売",
"データ通信サービス提供",
"ARアプリ開発",
"VRアプリ開発",
"VRコンテンツ制作",
"MRアプリ開発"]

#事業キーワードで検索
browser.find_element_by_xpath('//*[@id="root"]/div/div[3]/div[2]/div/div/div[1]/div[2]/div/div/div[1]/div/div/div/div/div[4]/button[2]').click()

for keyword in keylist:
    browser.find_element_by_xpath('/html/body/div[1]/div/div[4]/div[1]/div[5]/div/div/div/div[2]/div/div[1]/div/div[2]/div/input').clear()
    browser.find_element_by_xpath('/html/body/div[1]/div/div[4]/div[1]/div[5]/div/div/div/div[2]/div/div[1]/div/div[2]/div/input').send_keys(keyword)
    time.sleep(2)
    browser.find_element_by_xpath('//*[@id="modal-root"]/div[5]/div/div/div/div[2]/div/div[2]/div[1]/div[2]/div/label/input').click()
    time.sleep(1)

#ページ数確認
num=int(browser.find_element_by_xpath('//*[@id="modal-root"]/div[5]/div/div/div/div[3]/div[1]/div/div[2]').text)
if num%25==0:
    pages=round(num/25)
else:
    pages=round(num/25)+1
pages

browser.find_element_by_xpath('//*[@id="modal-root"]/div[5]/div/div/div/div[3]/div[2]/button[2]').click()
time.sleep(random_num(3))

#企業情報スクレイピング
database=[]
for i in range(pages):
    companies_name=browser.find_elements_by_xpath('//*[@class="searchListItem__head__info__heading_company_name"]')
    companies_keyword=browser.find_elements_by_xpath('//*[@class="searchListItem__keywords_right"]')
    companies_describe=browser.find_elements_by_xpath('//*[@class="searchListItem__desc"]')
    companies_address=browser.find_elements_by_xpath('//*[@class="searchListItem__head__info__lower flex"]')
    staff=browser.find_elements_by_xpath('//*[@id="root"]/div/div[3]/div[2]/div/div/div[2]/div/div/div[*]/div/div[@class="searchListItem__table flex"]/div[1]/div[1]/div[2]')
    year=browser.find_elements_by_xpath('//*[@id="root"]/div/div[3]/div[2]/div/div/div[2]/div/div/div[*]/div/div[@class="searchListItem__table flex"]/div[1]/div[2]/div[2]')
    money=browser.find_elements_by_xpath('//*[@id="root"]/div/div[3]/div[2]/div/div/div[2]/div/div/div[*]/div/div[@class="searchListItem__table flex"]/div[1]/div[3]/div[2]')                                                                                            
    area=browser.find_elements_by_xpath('//*[@id="root"]/div/div[3]/div[2]/div/div/div[2]/div/div/div[*]/div/div[@class="searchListItem__table flex"]/div[2]/div[1]/div[2]')
    area_sub=browser.find_elements_by_xpath('//*[@id="root"]/div/div[3]/div[2]/div/div/div[2]/div/div/div[*]/div/div[@class="searchListItem__table flex"]/div[2]/div[2]/div[2]')
    
    for a,b,c,d,i_staff,i_year,i_money,i_area,i_area_sub in zip(companies_name,companies_keyword,companies_describe,companies_address,staff,year,money,area,area_sub):
        b_clean="｜".join(re.findall(r'(?<=name=").+?(?="\>)', b.get_attribute("innerHTML")))
        area_clean="".join(re.findall(r'(?<=cell__).+?(?=</p>)', i_area.get_attribute("innerHTML"))).replace("smlcat\">"," 〇分業界：").replace("bgcat\">","●業界：").replace("&nbsp;,&nbsp;"," ")
        area_sub_clean="".join(re.findall(r'(?<=cell__).+?(?=</p>)',i_area_sub.get_attribute("innerHTML"))).replace("smlcat\">"," 〇分業界：").replace("bgcat\">","●業界：").replace("&nbsp;,&nbsp;"," ")
        des=c.text.split("\n")
        database.append([a.text,b_clean,des[0],des[1],d.text,i_staff.text,i_year.text,i_money.text,area_clean,area_sub_clean])
    browser.find_element_by_xpath('//*[@class="Pager__PagerButton-sc-1ytofcd-0 bAwGna"]').click()
    time.sleep(random_num(3))

browser.close()
    
df = pd.DataFrame(database,columns=["DB会社名","(3)事業内容キーワード","(1)要約","（2)概要","(23)住所","(11)従業員数","(6)設立年月","(8)資本金","(4)メイン業界","(5)サブ業界"])
df = df[["DB会社名","(1)要約","（2)概要","(3)事業内容キーワード","(4)メイン業界","(5)サブ業界","(6)設立年月","(8)資本金","(11)従業員数","(23)住所"]] 
df.to_excel('by_keyword.xlsx', sheet_name='sheet1')
df


,DB会社名,(1)要約,（2)概要,(3)事業内容キーワード,(4)メイン業界,(5)サブ業界,(6)設立年月,(8)資本金,(11)従業員数,(23)住所
0,ソフトバンク株式会社,インターネット関連および情報通信を行う会社,携帯端末の販売や、データ通信などの固定通信サービスを行う｢国内通信事業｣を行っている。また、...,決済代行｜スマホ・携帯電話卸売｜データ通信サービス提供｜スマホ・携帯電話販売,●業界：通信 〇分業界：通信回線,●業界：金融 〇分業界：クレジット・信販・決済代行,1986年12月,2043億900万円,1万7115人,東京都港区東新橋１丁目９番１号
1,株式会社エヌ・ティ・ティ・データ,官公庁などを対象としたシステムの開発および運用を行う会社,中央省庁や地方自治体を対象に税務や雇用および労働、貿易、交通などの情報システムの開発、運用を...,データ通信サービス提供｜クラウドサービス提供,●業界：IT 〇分業界：システム開発,●業界：IT 〇分業界：クラウド・フィンテック,1988年5月,1425億2000万円,1万1310人,東京都江東区豊洲３丁目３番３号
2,ＫＤＤＩ株式会社,個人向けの通信サービスや金融決済ならびにエンターテインメント等のサービスを提供する会社,通信・金融などのサービスを提供しており、パーソナル事業では、au・MVNO携帯電話、FTTH...,クラウドサービス提供｜データ通信サービス提供,●業界：通信 〇分業界：通信回線,●業界：IT 〇分業界：クラウド・フィンテック,1984年6月,1418億5200万円,1万968人,東京都新宿区西新宿２丁目３番２号
3,凸版印刷株式会社,印刷技術を基盤とした商品の製造および販売を行う会社,情報コミュニケーション事業として、通帳やＩＣカードなどのセキュア関連、カタログおよびパンフレ...,フィルム製造｜タイル製造｜翻訳ソフト開発｜電子決済システム開発｜ARアプリ開発｜パッケージデ...,●業界：広告 〇分業界：紙媒体印刷,●業界：IT 〇分業界：クラウド・フィンテック 〇分業界： その他IT 〇分業界： システム...,1900年,1049億8600万円,9993人,東京都台東区台東１丁目５番１号
4,ＮＥＣプラットフォームズ株式会社,ICTシステム機器の開発や製造や販売、設置および保守などを手掛ける会社,キーテレフォンやルーター等のICTシステム機器の開発や製造や販売、設置および保守などを行う。...,POSシステム開発｜ルーター（置き型）製造｜監視カメラシステム開発｜生産管理システム開発｜医...,●業界：機械 〇分業界：精密機器製造,●業界：IT 〇分業界：システム受託開発 〇分業界： システム開発●業界：コンサルティング ...,1932年11月,103億3100万円,6954人,神奈川県川崎市高津区北見方２丁目６番１号
5,エヌ・ティ・ティ・コミュニケーションズ株式会社,ネットワークやデータセンターの開発、構築などを行う会社,企業向けに、クラウドや専用回線を用いたネットワークおよびデータセンターなどの開発ならびに提案...,セキュリティサービス開発｜インターネット回線販売｜IP電話サービス提供｜マーケティング支援｜...,●業界：IT 〇分業界：システム開発,●業界：IT 〇分業界：情報セキュリティサービス 〇分業界： クラウド・フィンテック●業界：...,1999年7月,2309億円,5700人,東京都千代田区大手町2-3-1 大手町プレイスウエストタワー
6,コネクシオ株式会社,通信キャリア認定ショップの運営及び通信サービスの契約の取次などを行う会社,通信キャリア認定ショップを運営し、消費者や法人に対して、通信キャリアから供給される携帯電話の...,スマホ・携帯電話販売｜Webメディア運営｜スマホ周辺機器・スマホアクセサリー販売｜データ通信...,●業界：通信 〇分業界：携帯・通信回線販売代理店,●業界：IT 〇分業界：Webサービス・アプリ運営,1997年8月,27億7800万円,5222人,東京都新宿区西新宿８丁目１７番１号
7,東日本電信電話株式会社,東日本を対象とした通信サービスの提供を行う会社,東日本を対象に、電話やインターネット回線などといった通信サービスの提供を行う。また、法人の顧...,IP電話サービス提供｜インターネット回線販売｜データ通信サービス提供｜通信機器販売｜ネットワ...,●業界：通信 〇分業界：通信回線,●業界：IT 〇分業界：システム受託開発 〇分業界： ITインフラ,1999年7月,3350億円,4900人,東京都新宿区西新宿３丁目１９番２号
8,株式会社ティーガイア,個人や法人向けの通信サービスの契約取次および携帯電話などの販売を手掛ける会社,株式会社NTTドコモやソフトバンク株式会社など通信事業者の一次代理店として、通信サービスの利...,通信回線取次｜インターネット回線販売｜金券・ギフト券（商品券・旅行券・図書カード等）販売｜デ...,●業界：通信 〇分業界：携帯・通信回線販売代理店,●業界：小売 〇分業界：古本・リサイクルショップ●業界：金融 〇分業界：証券,1992年2月,31億5400万円,3725人,東京都渋谷区恵比寿４丁目１番１８号
9,グローリー株式会社,貨幣処理機や貨幣端末機、自動販売機などの製造・販売・保守を行う会社,金融機関の営業店において出納管理を行うオープン出納システムや、窓口用紙幣・硬貨入出金機、両替...,自動券売機製造｜両替機卸売｜両替機製造｜監視カメラシステム開発｜機械保守・メンテナンス｜紙幣...,●業界：機械 〇分業界：自動販売機・自動サービス機,●業界：製造 〇分業界：防災・防犯機器●業界：機械関連サービス 〇分業界：機械修理●業界：機...,1944年11月,128億9294万7600円,3447人,兵庫県姫路市下手野１丁目３番１号
